# Check device

In [ ]:
######################################################

# Check device
!nvidia-smi

Tue Apr  5 03:04:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Package Install

In [ ]:
# ====================================================
# Install
# ====================================================

!pip install -q datasets==1.18.3
!pip install -q sentencepiece==0.1.96
!pip install -q transformers==4.16.2

     |████████████████████████████████| 311 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 66.2 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.8 MB/s 
     |████████████████████████████████| 136 kB 74.7 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 144 kB 65.2 MB/s 
     |████████████████████████████████| 271 kB 73.6 MB/s 
     |████████████████████████████████| 1.2 MB 4.2 MB/s 
     |████████████████████████████████| 3.5 MB 4.1 MB/s 
     |████████████████████████████████| 895 kB 56.8 MB/s 
     |████████████████████████████████| 6.5 MB 55.5 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 


# Config

In [ ]:
# ====================================================
# CFG
# ====================================================
class CFG:
    env="colab"
    debug=False
    apex=True
    exp_name="nbme-exp073"
    competition_name="nbme-score-clinical-patient-notes"
    print_freq=100
    num_workers=4
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=15
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=1
    fc_dropout=0.2
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=8
    max_grad_norm=1000
    # MLM setting
    mlm_probability=0.15
    max_seq_length=512
    seed=42
    n_fold=1
    trn_fold=[0]
    train=True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [ ]:
# ====================================================
# Define path
# ====================================================
import sys
from pathlib import Path


print(CFG.env)
if CFG.env == "colab":
    # colab環境
    from google.colab import drive
    drive.mount("/content/drive")
    INPUT_DIR = Path("./drive/MyDrive/00.kaggle/input") / CFG.competition_name
    OUTPUT_DIR = Path("./drive/MyDrive/00.kaggle/output") / CFG.competition_name / CFG.exp_name
    if not OUTPUT_DIR.exists():
        OUTPUT_DIR.mkdir()

colab
Mounted at /content/drive


# Library

In [ ]:
# The following is necessary if you want to use the fast tokenizer for deberta v2 or v3
# This must be done before importing transformers
import shutil
from pathlib import Path

if CFG.env == "colab":
    input_dir = Path("./drive/MyDrive/00.kaggle/input/deberta-v2-3-fast-tokenizer")
    transformers_path = Path("/usr/local/lib/python3.7/dist-packages/transformers")
else:
    input_dir = Path("../input/deberta-v2-3-fast-tokenizer")
    transformers_path = Path("/opt/conda/lib/python3.7/site-packages/transformers")

convert_file = input_dir / "convert_slow_tokenizer.py"
conversion_path = transformers_path/convert_file.name

if conversion_path.exists():
    conversion_path.unlink()

shutil.copy(convert_file, transformers_path)
deberta_v2_path = transformers_path / "models" / "deberta_v2"

for filename in ['tokenization_deberta_v2.py', 'tokenization_deberta_v2_fast.py']:
    filepath = deberta_v2_path/filename
    if filepath.exists():
        filepath.unlink()

    shutil.copy(input_dir/filename, filepath)
    
    
from transformers.models.deberta_v2.tokenization_deberta_v2_fast import DebertaV2TokenizerFast

In [ ]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForMaskedLM
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers.modeling_outputs import MaskedLMOutput
from transformers import DataCollatorForLanguageModeling
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.11.6
transformers.__version__: 4.16.2
env: TOKENIZERS_PARALLELISM=true


# Utils

In [ ]:
# ====================================================
# Utils
# ===================================================
def get_logger(filename=OUTPUT_DIR/'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))

# Trainer

In [ ]:
######################################################

# Trainer
def trainer(model, data_loader, optimizer, scheduler, CFG):
    model.train()

    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = []

    optimizer.zero_grad()

    for idx, batch in enumerate(data_loader):
        for k, v in batch.items():
            batch[k] = v.to(device, dtype=torch.long)

        with torch.cuda.amp.autocast(enabled=CFG.apex):
            outputs = model(**batch)

        loss = outputs.loss

        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (idx + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            CFG.global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        
        losses.append(loss.detach().cpu().item())

        # if (CFG.global_step % CFG.save_step) == 0:
        #     LOGGER.info(
        #         "Epoch {} Step {}: Train Loss {:.4f}, elapsed {:.4f}s".format(
        #             CFG.epoch + 1, CFG.global_step, np.mean(losses), time.time() - start)
        #         )
        #     torch.save(
        #         model.state_dict(),
        #         OUTPUT_DIR + '{}-mlm-step-{}.bin'.format(
        #             CFG.model.replace('/', '-'),
        #             CFG.global_step))
        
        if idx % 500 == 0 or idx == (len(data_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss:.4f} '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, idx, len(data_loader), 
                          remain=timeSince(start, float(idx+1)/len(data_loader)),
                          loss=np.mean(losses),
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
    
    return np.mean(losses)

# Model

In [ ]:
def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
         'lr': encoder_lr, 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if "model" not in n],
         'lr': decoder_lr, 'weight_decay': 0.0}
    ]
    return optimizer_parameters

In [ ]:
def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler=='linear':
        scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler=='cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
    return scheduler

In [ ]:
# ====================================================
# Model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(
                cfg.model,
                output_hidden_states=False
                )
        else:
            self.config = torch.load(config_path)
        
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
            self.lm_head = AutoModelForMaskedLM.from_pretrained(cfg.model, config=self.config).cls # [cls, lm_head]
        else:
            self.model = AutoModel(self.config)
            self.lm_head = AutoModelForMaskedLM(self.config).cls # [cls, lm_head]
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(
            self, 
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            #position_ids=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None):
        
        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            #position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,)
        
        sequence_output = outputs[0]
        prediction_scores = self.lm_head(sequence_output)

        masked_lm_loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            masked_lm_loss = loss_fct(prediction_scores.view(-1, self.config.vocab_size), labels.view(-1))

        return MaskedLMOutput(loss=masked_lm_loss,
                              logits=prediction_scores,
                              hidden_states=outputs.hidden_states,
                              attentions=outputs.attentions)

# Main

In [ ]:
train = pd.read_csv(INPUT_DIR / "train.csv")
features = pd.read_csv(INPUT_DIR / "features.csv")
patient_notes = pd.read_csv(INPUT_DIR / "patient_notes.csv")
test = pd.read_csv(INPUT_DIR / "test.csv")

train.shape, features.shape, patient_notes.shape, test.shape

((14300, 6), (143, 3), (42146, 3), (5, 4))

In [ ]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = DebertaV2TokenizerFast.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR / "tokenizer/")
CFG.tokenizer = tokenizer

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tmp = 'dad with recent heart attack'
encode = tokenizer(tmp, return_offsets_mapping=True)
for (start,end) in encode['offset_mapping']:
    print(f"'{tmp[start:end]}', {start}, {end}")

print("ans")
print("""
'', 0, 0
'dad', 0, 3
' with', 3, 8
' recent', 8, 15
' heart', 15, 21
' attack', 21, 28
'', 0, 0
""")

'', 0, 0
'dad', 0, 3
' with', 3, 8
' recent', 8, 15
' heart', 15, 21
' attack', 21, 28
'', 0, 0
ans

'', 0, 0
'dad', 0, 3
' with', 3, 8
' recent', 8, 15
' heart', 15, 21
' attack', 21, 28
'', 0, 0



In [ ]:
mlm_data = patient_notes[['pn_history']].reset_index(drop=True)
mlm_data = mlm_data.rename(columns={'pn_history': 'text'})
csv_name = f'mlm_data.csv'
mlm_data.to_csv(OUTPUT_DIR / csv_name, index=False)
print(f"Saved mlm data: {csv_name}")
print(f"mlm data: {mlm_data.shape}")

Saved mlm data: mlm_data.csv
mlm data: (42146, 1)


In [ ]:
#####################################################

# Training support

def tokenize_function(examples):
    return tokenizer(examples["text"], return_special_tokens_mask=True)

def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // CFG.max_len) * CFG.max_len
    result = {
        k: [t[i : i + CFG.max_len] for i in range(0, total_length, CFG.max_len)]
        for k, t in concatenated_examples.items()
    }
    return result

In [ ]:
seed_everything(CFG.seed)

CFG.train_file = f"mlm_data.csv"
data_files = {'train': str(OUTPUT_DIR / CFG.train_file)}
raw_datasets = load_dataset('csv', data_files=data_files)

Using custom data configuration default-361353aaab79983e


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-361353aaab79983e/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
if CFG.max_seq_length is None:
    max_seq_length = tokenizer.model_max_length
else:
    if CFG.max_seq_length > tokenizer.model_max_length:
        max_seq_length = min(CFG.max_seq_length, tokenizer.model_max_length)

In [ ]:
tokenizer.model_max_length

1000000000000000019884624838656

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    remove_columns=["text"],
    load_from_cache_file=not True,
    )
LOGGER.info(f"tokenized_datasets: {tokenized_datasets}")

Setting TOKENIZERS_PARALLELISM=false for forked processes.
tokenized_datasets: DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 42146
    })
})


In [ ]:
tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    num_proc=4,
    load_from_cache_file=not True,
    )
train_dataset = tokenized_datasets["train"]
LOGGER.info(f"train_dataset: {train_dataset}")

Setting TOKENIZERS_PARALLELISM=false for forked processes.
train_dataset: Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
    num_rows: 15622
})


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm_probability=CFG.mlm_probability
    )
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=CFG.batch_size
    )

In [ ]:
model = CustomModel(CFG, config_path=None, pretrained=True)
model.to(device)
optimizer_parameters = get_optimizer_params(
    model,
    encoder_lr=CFG.encoder_lr, 
    decoder_lr=CFG.decoder_lr,
    weight_decay=CFG.weight_decay)
optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

Downloading:   0%|          | 0.00/833M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the 

In [ ]:
num_train_steps = int(len(mlm_data) / CFG.batch_size * CFG.epochs)
scheduler = get_scheduler(CFG, optimizer, num_train_steps)

In [ ]:
print(num_train_steps)

632190


In [27]:
CFG.global_step = 0
CFG.save_step = 25000

for epoch in range(CFG.epochs):
    CFG.epoch = epoch
    start = time.time()
        
    train_loss = trainer(model, train_loader, optimizer, scheduler, CFG)

    LOGGER.info(
        "Epoch {}: Train Loss {:.4f}, elapsed {:.4f}s".format(
            epoch + 1, train_loss, time.time() - start)
        )
    torch.save(
        model.state_dict(),
        str(OUTPUT_DIR / '{}-mlm-epoch-{}.bin'.format(
            CFG.model.replace('/', '-'),
            epoch + 1))
        )

Epoch: [1][0/15622] Elapsed 0m 1s (remain 443m 55s) Loss: 1.5002 Grad: 355674.5312  LR: 0.00002000  
Epoch: [1][500/15622] Elapsed 1m 59s (remain 60m 15s) Loss: 1.0712 Grad: 104145.1016  LR: 0.00002000  
Epoch: [1][1000/15622] Elapsed 3m 44s (remain 54m 35s) Loss: 0.9778 Grad: 112734.4531  LR: 0.00002000  
Epoch: [1][1500/15622] Elapsed 5m 18s (remain 49m 59s) Loss: 0.9181 Grad: 134490.7188  LR: 0.00002000  
Epoch: [1][2000/15622] Elapsed 6m 52s (remain 46m 48s) Loss: 0.8735 Grad: 135526.1875  LR: 0.00002000  
Epoch: [1][2500/15622] Elapsed 8m 26s (remain 44m 16s) Loss: 0.8389 Grad: 111095.7656  LR: 0.00002000  
Epoch: [1][3000/15622] Elapsed 9m 59s (remain 42m 1s) Loss: 0.8094 Grad: 136808.8750  LR: 0.00002000  
Epoch: [1][3500/15622] Elapsed 11m 32s (remain 39m 58s) Loss: 0.7843 Grad: 125526.3203  LR: 0.00002000  
Epoch: [1][4000/15622] Elapsed 13m 6s (remain 38m 4s) Loss: 0.7619 Grad: 130371.5781  LR: 0.00002000  
Epoch: [1][4500/15622] Elapsed 14m 39s (remain 36m 13s) Loss: 0.7427 

Epoch 1: Train Loss 0.5413, elapsed 2934.7519s


Epoch: [1][15621/15622] Elapsed 48m 54s (remain 0m 0s) Loss: 0.5413 Grad: 124059.5234  LR: 0.00002000  
Epoch: [2][0/15622] Elapsed 0m 0s (remain 67m 19s) Loss: 0.3377 Grad: 114214.3906  LR: 0.00002000  
Epoch: [2][500/15622] Elapsed 1m 38s (remain 49m 47s) Loss: 0.3842 Grad: 181291.1094  LR: 0.00002000  
Epoch: [2][1000/15622] Elapsed 3m 11s (remain 46m 35s) Loss: 0.3808 Grad: 104461.5156  LR: 0.00002000  
Epoch: [2][1500/15622] Elapsed 4m 43s (remain 44m 28s) Loss: 0.3766 Grad: 112049.0625  LR: 0.00002000  
Epoch: [2][2000/15622] Elapsed 6m 15s (remain 42m 37s) Loss: 0.3756 Grad: 139150.2344  LR: 0.00002000  
Epoch: [2][2500/15622] Elapsed 7m 47s (remain 40m 54s) Loss: 0.3748 Grad: 108020.1328  LR: 0.00002000  
Epoch: [2][3000/15622] Elapsed 9m 20s (remain 39m 16s) Loss: 0.3738 Grad: 123903.1484  LR: 0.00002000  
Epoch: [2][3500/15622] Elapsed 10m 53s (remain 37m 40s) Loss: 0.3722 Grad: 113182.0391  LR: 0.00002000  
Epoch: [2][4000/15622] Elapsed 12m 27s (remain 36m 10s) Loss: 0.3703

Epoch 2: Train Loss 0.3388, elapsed 2926.8347s


Epoch: [2][15621/15622] Elapsed 48m 46s (remain 0m 0s) Loss: 0.3388 Grad: 107116.8984  LR: 0.00002000  
Epoch: [3][0/15622] Elapsed 0m 0s (remain 61m 1s) Loss: 0.3111 Grad: 93935.1719  LR: 0.00002000  
Epoch: [3][500/15622] Elapsed 1m 41s (remain 51m 10s) Loss: 0.3023 Grad: 102337.0234  LR: 0.00002000  
Epoch: [3][1000/15622] Elapsed 3m 16s (remain 47m 44s) Loss: 0.3011 Grad: 99505.4375  LR: 0.00002000  
Epoch: [3][1500/15622] Elapsed 4m 50s (remain 45m 34s) Loss: 0.3010 Grad: 80483.4453  LR: 0.00002000  
Epoch: [3][2000/15622] Elapsed 6m 25s (remain 43m 45s) Loss: 0.3008 Grad: 115001.9453  LR: 0.00002000  
Epoch: [3][2500/15622] Elapsed 7m 59s (remain 41m 57s) Loss: 0.3002 Grad: 111458.1719  LR: 0.00002000  
Epoch: [3][3000/15622] Elapsed 9m 35s (remain 40m 18s) Loss: 0.2992 Grad: 109338.2266  LR: 0.00002000  
Epoch: [3][3500/15622] Elapsed 11m 8s (remain 38m 35s) Loss: 0.2986 Grad: 125289.1094  LR: 0.00002000  
Epoch: [3][4000/15622] Elapsed 12m 43s (remain 36m 57s) Loss: 0.2977 Grad

...

Epoch 11: Train Loss 0.1798, elapsed 2903.1409s


Epoch: [11][15621/15622] Elapsed 48m 23s (remain 0m 0s) Loss: 0.1798 Grad: 86432.0391  LR: 0.00001994  
Epoch: [12][0/15622] Elapsed 0m 0s (remain 71m 31s) Loss: 0.1019 Grad: 67177.0000  LR: 0.00001994  
Epoch: [12][500/15622] Elapsed 1m 41s (remain 51m 8s) Loss: 0.1800 Grad: 132893.7812  LR: 0.00001994  
Epoch: [12][1000/15622] Elapsed 3m 13s (remain 47m 5s) Loss: 0.1783 Grad: 129994.8438  LR: 0.00001994  
Epoch: [12][1500/15622] Elapsed 4m 45s (remain 44m 44s) Loss: 0.1775 Grad: 76830.8984  LR: 0.00001994  
Epoch: [12][2000/15622] Elapsed 6m 17s (remain 42m 52s) Loss: 0.1775 Grad: 109596.2734  LR: 0.00001994  
Epoch: [12][2500/15622] Elapsed 7m 48s (remain 40m 59s) Loss: 0.1770 Grad: 83541.4531  LR: 0.00001994  
Epoch: [12][3000/15622] Elapsed 9m 21s (remain 39m 19s) Loss: 0.1769 Grad: 79250.1094  LR: 0.00001994  
Epoch: [12][3500/15622] Elapsed 10m 52s (remain 37m 38s) Loss: 0.1769 Grad: 79704.7891  LR: 0.00001994  
Epoch: [12][4000/15622] Elapsed 12m 24s (remain 36m 3s) Loss: 0.177

Epoch 12: Train Loss 0.1792, elapsed 2886.6191s


Epoch: [12][15621/15622] Elapsed 48m 6s (remain 0m 0s) Loss: 0.1792 Grad: 40001.1602  LR: 0.00001993  
Epoch: [13][0/15622] Elapsed 0m 0s (remain 63m 50s) Loss: 0.1742 Grad: 77735.8281  LR: 0.00001993  
Epoch: [13][500/15622] Elapsed 1m 43s (remain 51m 52s) Loss: 0.1768 Grad: 84145.0156  LR: 0.00001993  
Epoch: [13][1000/15622] Elapsed 3m 16s (remain 47m 49s) Loss: 0.1757 Grad: 91087.8984  LR: 0.00001993  
Epoch: [13][1500/15622] Elapsed 4m 49s (remain 45m 21s) Loss: 0.1747 Grad: 95914.7656  LR: 0.00001993  
Epoch: [13][2000/15622] Elapsed 6m 22s (remain 43m 22s) Loss: 0.1741 Grad: 107993.8828  LR: 0.00001993  
Epoch: [13][2500/15622] Elapsed 7m 55s (remain 41m 34s) Loss: 0.1728 Grad: 92399.5078  LR: 0.00001993  
Epoch: [13][3000/15622] Elapsed 9m 28s (remain 39m 51s) Loss: 0.1730 Grad: 90549.5156  LR: 0.00001993  
Epoch: [13][3500/15622] Elapsed 11m 1s (remain 38m 8s) Loss: 0.1737 Grad: 103059.1328  LR: 0.00001993  
Epoch: [13][4000/15622] Elapsed 12m 34s (remain 36m 32s) Loss: 0.1739

KeyboardInterrupt: ignored